In [33]:
import numpy as np
import onnxruntime as ort
from sentence_transformers import SentenceTransformer

In [34]:
onnx_model_path = "cnn1d_sentiment.onnx"
session = ort.InferenceSession(onnx_model_path)

In [35]:
embedder = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-cos-v1')

In [36]:
def predict_sentiment(text):
    # Get embedding
    embedding = embedder.encode([text])  # shape: (1, 768)
    embedding = embedding.astype(np.float32)

    # Run through ONNX model
    ort_inputs = {session.get_inputs()[0].name: embedding}
    ort_outs = session.run(None, ort_inputs)
    logits = ort_outs[0].squeeze()  # shape: ()
    prob = 1 / (1 + np.exp(-logits))  # sigmoid

    # Binary prediction
    pred = int(prob > 0.5)
    return pred, float(prob)

In [37]:
text = "I don't like this movie."
pred, prob = predict_sentiment(text)
print(f"Sentiment: {pred} ({'positive' if pred == 1 else 'negative'}) (probability: {prob:.4f})")

Sentiment: 0 (negative) (probability: 0.1454)
